# Цель 
- проанализировать данные для развития нового продукта, сервиса для чтения книг


Задачи:
- Посчитайте, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитайте количество обзоров и среднюю оценку;
- Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
- Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
- Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

Импортируем библиотеки

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

Выведем первые строки каждой таблицы

In [3]:
# чтобы выполнить SQL-запрос, используем Pandas
query = '''SELECT * FROM books LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [4]:
authors = """SELECT *
FROM authors 
LIMIT 5"""
pd.io.sql.read_sql(sql=text(authors), con = con)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [5]:
publishers = """SELECT *
FROM publishers 
LIMIT 5"""
pd.io.sql.read_sql(sql=text(publishers), con = con)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [6]:
ratings = """SELECT *
FROM ratings 
LIMIT 5"""
pd.io.sql.read_sql(sql=text(ratings), con = con)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [7]:
reviews = """SELECT *
FROM reviews 
LIMIT 5"""
pd.io.sql.read_sql(sql=text(reviews), con = con)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


Задания

- Посчитайте, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитайте количество обзоров и среднюю оценку;
- Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
- Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
- Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

# Посчитаем, сколько книг вышло после 1 января 2000 года

In [8]:
cnt_books = """SELECT count(*) 
FROM books 
WHERE publication_date > '2000-01-01'"""
pd.io.sql.read_sql(sql=text(cnt_books), con = con)

,count
0,819


После 1 января 2000 года вышло всего = 819 книг

# Для каждой книги посчитаем количество обзоров и среднюю оценку

In [9]:
review_rating = """
WITH 
cnt_rev AS ( SELECT book_id
                    , count(review_id) as cnt_reviews
                 FROM reviews 
                 GROUP BY book_id ),
          
avg_rat AS (SELECT book_id
                   , AVG(rating) as avg_rating
                 FROM ratings 
                 GROUP BY book_id )
              
SELECT 
         cnt_rev.book_id
        , b.title
        , cnt_reviews
        , avg_rating       
FROM books AS b 
LEFT JOIN cnt_rev ON b.book_id = cnt_rev.book_id
LEFT JOIN avg_rat ON b.book_id = avg_rat.book_id

"""
pd.io.sql.read_sql(sql=text(review_rating), con = con)

,book_id,title,cnt_reviews,avg_rating
0,652.0,The Body in the Library (Miss Marple #3),2.0,4.500000
1,273.0,Galápagos,2.0,4.500000
2,51.0,A Tree Grows in Brooklyn,5.0,4.250000
3,951.0,Undaunted Courage: The Pioneering First Missio...,2.0,4.000000
4,839.0,The Prophet,4.0,4.285714
...,...,...,...,...
995,64.0,Alice in Wonderland,4.0,4.230769
996,55.0,A Woman of Substance (Emma Harte Saga #1),2.0,5.000000
997,148.0,Christine,3.0,3.428571
998,790.0,The Magicians' Guild (Black Magician Trilogy #1),2.0,3.500000


в таблице представленны данные по обзорам и рейтингам книг где: 
- book_id - индификатор книги
- title - название книги
- cnt_reviews - количество обзоров книги
- avg_rating - средняя оценка книги

# Определим издательство, которое выпустило наибольшее число книг толще 50 страниц — так исключим из анализа брошюры

In [10]:
publisher_max = """
SELECT p.publisher_id
       , p.publisher
       , COUNT(b.book_id) as cnt_book
FROM books AS b
JOIN publishers AS p ON p.publisher_id = b.publisher_id
WHERE b.num_pages > '50'
GROUP BY p.publisher_id
ORDER BY cnt_book DESC
LIMIT 1
"""
pd.io.sql.read_sql(sql=text(publisher_max), con = con)

,publisher_id,publisher,cnt_book
0,212,Penguin Books,42


Издательство Penguin Books выпустило больше всего книг

# Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;

In [11]:
author_max = """
SELECT  b.author_id
       , a.author
       , AVG(r.rating) AS avg_rating
FROM books AS b
JOIN authors AS a ON a.author_id = b.author_id
JOIN ratings AS r ON b.book_id = r.book_id
WHERE b.book_id IN ( SELECT book_id    
                   FROM ratings
                   GROUP BY book_id
                   HAVING COUNT(rating) >= 50 )
GROUP BY b.author_id, a.author
ORDER BY avg_rating desc
LIMIT 1

"""
pd.io.sql.read_sql(sql=text(author_max), con = con)

,author_id,author,avg_rating
0,236,J.K. Rowling/Mary GrandPré,4.287097


Автор с самой высокой средней оценкой книг - J.K. Rowling/Mary GrandPré

# Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

In [12]:
agv_rev_48 = """
WITH 
t1 AS ( SELECT username,
                    COUNT(review_id) AS cnt_review
             FROM reviews
             WHERE username IN (SELECT username
                                FROM ratings
                                GROUP BY username
                                HAVING COUNT(rating_id) > 48)
                                GROUP BY username )
          

              
SELECT AVG(cnt_review)    
FROM t1 

"""
pd.io.sql.read_sql(sql=text(agv_rev_48), con = con)

,avg
0,24.0


Пользователи которые поставили больше 48 оценок в среднем делали по 24 обзора 